Prediction Model Data

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)


/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [2]:
def get_lags(data, x, lag, f=None):
    lag_avg = pd.DataFrame(data[x])

    def assign_na(x):
        try:
            y = np.float64(x)
        except:
            y = np.NaN
        return y

    lag_avg[x] = lag_avg[x].apply(assign_na)

    for i in range(1, lag):
        lag_avg['lag_{}'.format(i)] = lag_avg[x].copy().shift(i)

    ten_day_avg = []
    for i, row in lag_avg.iterrows():
        if f == 'sum':
            ten_day_avg.append(row.sum(skipna=True))
        else:
            ten_day_avg.append(row.mean(skipna=True))

    return pd.Series(ten_day_avg)

In [3]:
spray_cost = 5200  # per square mile
hospitalization_cost = 13810  # total acute medical care "Initial and Long-Term Costs of Patients Hospitalized with WNV Disease

hospitalization_rate = .49

chicago_total_cost = 425000

In [4]:
df_in = pd.read_csv('data/train.csv')

df_in.shape

(10506, 12)

In [5]:
zip_codes = pd.read_csv('data/WNV_Pop_Age.csv')

zip_codes = zip_codes[['Trap', 'ZipCode']].rename(columns={'Trap': 'trap', 'ZipCode': 'zip_code'})

In [6]:
df_out = df_in.join(zip_codes)

In [7]:
df_out.head(1)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,trap,zip_code
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0,T002,60634


In [8]:
demographics = pd.read_csv('data/demographics.csv').drop('Unnamed: 0', axis=1)

In [9]:
demographics.head()

,zip_code,percent_employed,percent_no_health,percent_public_health,total_population,population_porportion,percent_age_over_65,percent_age_under_14,percent_low_income
0,60002,69.500000,9.4,22.400000,24250.0,0.001887,12.000000,19.299999,8.7
1,60004,68.000000,8.7,22.600000,49957.0,0.003888,17.100000,18.000000,4.3
2,60005,68.199997,10.6,23.600000,30931.0,0.002407,16.100000,19.400002,7.2
3,60007,70.199997,8.7,22.100000,33973.0,0.002644,14.900001,16.000000,5.6
4,60008,69.599998,19.5,26.200001,22302.0,0.001735,12.700000,19.500000,8.9


In [10]:
dfsss = df_out.merge(demographics, on=['zip_code'])

dfsss.drop('trap', axis=1, inplace=True)

to_drop = ['Address', 'Species', 'Block', 'Street', 'AddressNumberAndStreet', 'AddressAccuracy', 'WnvPresent', 'Trap']

dfsss = dfsss.drop(to_drop, axis=1);

In [11]:
dfsss.head(1)

,Date,Latitude,Longitude,NumMosquitos,zip_code,percent_employed,percent_no_health,percent_public_health,total_population,population_porportion,percent_age_over_65,percent_age_under_14,percent_low_income
0,2007-05-29,41.95469,-87.800991,1,60634,68.0,21.9,29.9,72718.0,0.005659,12.799999,17.200001,10.5


In [12]:
geo = pd.read_csv('data/WNV_Pop_Age_Temp_Geog.csv')
geo = geo[['ZipCode', 'LandArea', 'WaterArea']]
geo = geo.groupby('ZipCode').mean().reset_index().rename(columns={'ZipCode': 'zip_code'})

In [13]:
geo.head()

,zip_code,LandArea,WaterArea
0,60018,16.3,0.3
1,60106,9.0,0.1
2,60605,1.2,0.8
3,60607,2.3,0.0
4,60608,6.4,0.2


In [14]:
dfsss = dfsss.merge(geo, on='zip_code')

dfsss.rename(columns={'LandArea': 'land_area', 'WaterArea': 'water_area'}, inplace=True)

In [15]:
dfsss.head()

,Date,Latitude,Longitude,NumMosquitos,zip_code,percent_employed,percent_no_health,percent_public_health,total_population,population_porportion,percent_age_over_65,percent_age_under_14,percent_low_income,land_area,water_area
0,2007-05-29,41.95469,-87.800991,1,60634,68.0,21.9,29.9,72718.0,0.005659,12.799999,17.200001,10.5,7.1,0.0
1,2007-05-29,41.95469,-87.800991,1,60634,68.0,21.9,29.9,72718.0,0.005659,12.799999,17.200001,10.5,7.1,0.0
2,2007-06-05,41.95469,-87.800991,3,60634,68.0,21.9,29.9,72718.0,0.005659,12.799999,17.200001,10.5,7.1,0.0
3,2007-06-05,41.95469,-87.800991,5,60634,68.0,21.9,29.9,72718.0,0.005659,12.799999,17.200001,10.5,7.1,0.0
4,2007-06-05,41.95469,-87.800991,1,60634,68.0,21.9,29.9,72718.0,0.005659,12.799999,17.200001,10.5,7.1,0.0


In [16]:
to_sum = dfsss[['Date', 'zip_code', 'NumMosquitos']]

summed = to_sum.groupby(['Date', 'zip_code']).sum().reset_index()

In [17]:
to_mean = dfsss[['Date', 'zip_code', 'Latitude', 'Longitude']]

meaned = to_mean.groupby(['Date', 'zip_code']).mean().reset_index()

In [18]:
df = meaned.merge(summed, on=['Date', 'zip_code'])
df = df.merge(dfsss, on=['Date', 'zip_code'], how='left')
df = df.drop(['Latitude_y', 'Longitude_y', 'NumMosquitos_y'], axis=1)
df = df.rename(columns={'Date': 'date', 'Latitude_x': 'latitude', 'Longitude_x': 'longitude', 'NumMosquitos_x': 'num_mosquitos'})

df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['week'] = df['date'].apply(lambda x: x.isocalendar()[1])

In [19]:
df.head(3)

,date,zip_code,latitude,longitude,num_mosquitos,percent_employed,percent_no_health,percent_public_health,total_population,population_porportion,percent_age_over_65,percent_age_under_14,percent_low_income,land_area,water_area,year,week
0,2007-05-29,60608,41.865503,-87.652436,4,60.900002,29.5,36.099998,78794.0,0.006132,7.7,18.400002,19.5,6.4,0.2,2007,22
1,2007-05-29,60608,41.865503,-87.652436,4,60.900002,29.5,36.099998,78794.0,0.006132,7.7,18.400002,19.5,6.4,0.2,2007,22
2,2007-05-29,60608,41.865503,-87.652436,4,60.900002,29.5,36.099998,78794.0,0.006132,7.7,18.400002,19.5,6.4,0.2,2007,22


In [20]:
wnv_years = []
for i in range(2002,2017):
    wnv_years.append(str(i))
wnv_cases = [635, 20, 23, 135, 29, 23, 9, 1, 30, 22, 174, 60, np.NaN, 27, 90]
wnv_deaths = [42, 1, 2, 6, np.NaN, 0, 0, 0, 1, 1, 5, 7, np.NaN, 0, 0]

deaths = pd.DataFrame({"Year": wnv_years, "Cases": wnv_cases, "Deaths" : wnv_deaths})
deaths = deaths.set_index('Year')
deaths = deaths.shift(1).reset_index()
deaths = deaths.rename(columns={'Year': 'year', 'Cases': 'averaged_cases', 'Deaths': 'averaged_deaths'})
deaths['year'] = deaths['year'].astype(np.int64)


In [21]:
deaths['averaged_cases'] = get_lags(deaths, 'averaged_cases', 5, np.mean)
deaths['averaged_deaths'] = get_lags(deaths, 'averaged_deaths', 5, np.mean)

In [22]:
df = df.merge(deaths, on='year')


In [23]:
df.head(1)

,date,zip_code,latitude,longitude,num_mosquitos,percent_employed,percent_no_health,percent_public_health,total_population,population_porportion,percent_age_over_65,percent_age_under_14,percent_low_income,land_area,water_area,year,week,averaged_cases,averaged_deaths
0,2007-05-29,60608,41.865503,-87.652436,4,60.900002,29.5,36.099998,78794.0,0.006132,7.7,18.400002,19.5,6.4,0.2,2007,22,168.4,12.75


In [24]:
df['at_risk_population_percent'] = df['percent_age_over_65'] + df['percent_age_under_14']
df['at_risk_population'] = df['at_risk_population_percent']/100 * df['total_population'] 

df['proportional_averaged_cases'] = df['population_porportion'] * df['averaged_cases']
df['proportional_averaged_deaths'] = df['population_porportion'] * df['averaged_deaths']


df['high_risk_infection_percent'] = df['averaged_cases'] * df['at_risk_population_percent']/100

In [25]:
df = df.groupby(['date', 'zip_code']).mean().reset_index()

In [26]:
to_drop = ['percent_employed', 'percent_no_health', 'percent_public_health',
           'percent_age_over_65', 'percent_age_under_14', 'percent_low_income']

df = df.drop(to_drop, axis=1)

In [27]:
predictions = pd.read_csv('data/prediction_probabilities.csv')
predictions['Id'] -= 1
predictions = predictions.set_index('Id')

In [28]:
df = df.join(predictions)

In [29]:
# Estimate Costs...

df['cost_to_spray'] = spray_cost * df['land_area']

In [30]:
df[(df['year'] == 2013) & (df['week'] == 23)].head()

,date,zip_code,latitude,longitude,num_mosquitos,total_population,population_porportion,land_area,water_area,year,week,averaged_cases,averaged_deaths,at_risk_population_percent,at_risk_population,proportional_averaged_cases,proportional_averaged_deaths,high_risk_infection_percent,wnv_probablitiy,cost_to_spray
1989,2013-06-07,60018,41.992478,-87.862995,16,28749.0,0.002237,16.3,0.3,2013,23,47.2,1.4,33.000000,9487.170000,0.105595,0.003132,15.576000,0.357962,84760.0
1990,2013-06-07,60106,41.957799,-87.930995,32,20276.0,0.001578,9.0,0.1,2013,23,47.2,1.4,31.100000,6305.836077,0.074474,0.002209,14.679200,0.357962,46800.0
1991,2013-06-07,60608,41.867108,-87.654224,2,78794.0,0.006132,6.4,0.2,2013,23,47.2,1.4,26.100002,20565.235428,0.289410,0.008584,12.319201,0.357962,33280.0
1992,2013-06-07,60609,41.812418,-87.663779,13,62773.0,0.004885,7.8,0.0,2013,23,47.2,1.4,31.899998,20024.585863,0.230565,0.006839,15.056799,0.357962,40560.0
1993,2013-06-07,60612,41.868142,-87.696269,2,36052.0,0.002805,3.7,0.0,2013,23,47.2,1.4,29.700000,10707.443931,0.132419,0.003928,14.018400,0.331111,19240.0


In [31]:
# If we sprayed everywhere, every week....

df['cost_to_spray'].sum()

70458439.99999994

In [32]:
df['week'].nunique()

20

In [33]:
df['spray_metric'] = df['wnv_probablitiy'] * df['at_risk_population'] / df['cost_to_spray']

In [34]:
df.to_csv('data/recommendation_data.csv')

In [35]:
df_work = df[(df['year'] == 2013)]
df_work['spray_metric'].describe()

for i in range(1, 11):
    print(i/10, df_work[df_work['spray_metric'] > i/10]['cost_to_spray'].sum())


0.1 14725880.0
0.2 9628840.0
0.3 4669600.0
0.4 1664520.0
0.5 279240.0
0.6 nan
0.7 nan
0.8 nan
0.9 nan
1.0 nan


In [36]:
# Agressive Spraying
df_work[df_work['spray_metric'] > 0.48]['cost_to_spray'].sum()

358800.00000000023

In [37]:
df_work[df_work['spray_metric'] > 0.48]['land_area'].sum()

69.000000000000043

In [38]:
df_work[df_work['spray_metric'] > 0.48];

In [39]:
# Moderate Spraying
df_work[df_work['spray_metric'] > 0.5]['cost_to_spray'].sum()

279240.00000000017

In [40]:
df_work[df_work['spray_metric'] > 0.5]['land_area'].sum()

53.700000000000031

In [41]:
df_work[df_work['spray_metric'] > 0.5];

In [42]:
# Conservative Spraying
df_work[df_work['spray_metric'] > 0.52]['cost_to_spray'].sum()

126359.99999999999

In [43]:
df_work[df_work['spray_metric'] > 0.52]['land_area'].sum()

24.299999999999986

In [44]:
df_work[df_work['spray_metric'] > 0.52];